In [4]:
import pandas as pd

import os
from datetime import datetime
import numpy as np
def evaluate_trading_metrics(trades: pd.DataFrame, pnl_col: str = 'P&L') -> dict:
    """
    Oblicza podstawowe metryki strategii tradingowej:
    - Profit Factor
    - Łączne P&L
    - Sharpe Ratio

    trades : pd.DataFrame
        Tabela z historią transakcji, zawierająca kolumnę P&L.
    pnl_col : str
        Nazwa kolumny z zyskiem/stratą dla każdej transakcji.

    Returns:
        dict z wynikami metryk
    """

    pnl = trades[pnl_col]

    # Profit Factor
    total_profit = pnl[pnl > 0].sum()
    total_loss = pnl[pnl < 0].sum()
    profit_factor = total_profit / abs(total_loss) if total_loss != 0 else np.inf

    # Total P&L
    total_pnl = pnl.sum()


    return {
        'Profit Factor': round(profit_factor, 4),
        'P&L': round(total_pnl, 2),
        'Number': len(trades)
    }

def calculate_max_drawdown(account_history: pd.DataFrame, value_col: str = 'account_value'):
    """
    Calculates the maximum drawdown in percentage.


    Paramete
    Parameters:
    - account_history: DataFrame z kolumną wartości konta
    - value_col: nazwa kolumny z wartością konta

    Returns:
    - Max Drawdown w % (ujemna wartość)
    """

    values = account_history[value_col]
    peak = values.cummax()
    drawdown = (values - peak) / peak
    max_drawdown = drawdown.min()  # najbardziej negatywna wartość
    return round(max_drawdown * 100, 2), round(values.iloc[-1] - values[0], 2)   # dropdown w procentach, oraz końcowy P&L

In [5]:

def generate_simulation_summary(trades, history):
    metrics = evaluate_trading_metrics(trades)
    metrics['Max Drawdown'], metrics['P&L after fees'] = calculate_max_drawdown(history)
    # print(f"Number: {metrics['Number']}")
    # print(f"Profit Factor: {metrics['Profit Factor']}")    
    # print(f"P&L: {metrics['P&L']}")
    # print(f"P&L after fees {metrics['P&L after fees']}")
    # print(f"Max Drawdown: {metrics['Drop Down']}%")
    return metrics

# analyze_simulation_output(trades, history)

In [ ]:

# Konstruowanie nazw plików
ordered_cols = ['Strategy', 'Description', 'Ticker', 'Interval', 'Profit Factor', 'P&L', 'Number', 'P&L after fees', 'Max Drawdown']
def calculate_roi(pl, days, initial_deposit):
    return round(pl/initial_deposit/(days/365)*100,2)
def create_new_row(strategy, description, ticker, interval, metrics,
                   start_date, end_date,  initial_deposit, category, okres_id):
    metrics['Strategy'] = strategy
    metrics['Interval'] = interval
    metrics['Ticker'] = ticker
    metrics['Description'] = description
    metrics['Start_date'] = start_date
    metrics['End_date'] = end_date
    metrics['length(d)'] = (end_date - start_date).days
    metrics['ROI(% year)'] = calculate_roi(metrics['P&L after fees'], metrics['length(d)'] , initial_deposit)
    metrics['Deposit'] = initial_deposit
    metrics['Category'] = category
    metrics['Okres'] = okres_id
    all_cols = ordered_cols + extra_cols
    return pd.DataFrame([metrics])[all_cols]

def upload_row(df, idx, start_date, end_date, initial_deposit, category, okres_id):
    df.loc[idx, 'Start_date']  = start_date
    df.loc[idx, 'End_date']= end_date
    df.loc[idx, 'length(d)'] = (end_date - start_date).days
    df.loc[idx, 'ROI(% year)'] = calculate_roi(df.loc[idx, 'P&L after fees'], df.loc[idx, 'length(d)'], initial_deposit)
    df.loc[idx, 'Category'] = category
    df.loc[idx, 'Deposit'] = initial_deposit
    df.loc[idx, 'Okres'] = okres_id
    return df

dane_gieldowe = pd.read_excel('Market_data.xlsx')

strategies =pd.read_excel('Strategies_data.xlsx')
# 
# print(dane_gieldowe.head())



simulation_data='output_simulations'


folder_data_input='row_data'
folder_output='output_simulations'
# 
start_date = '2025-01-01'
#start_date = '2025-01-21'
start_date="2023-08-01"
okres_id = '3'

# Pełna ścieżka do nowego folderu
output_path = os.path.join(folder_output, start_date)

os.makedirs(folder_output ,exist_ok=True)
# Tworzenie folderu jeśli nie istnieje
os.makedirs(output_path, exist_ok=True)

#strategy='0.4'
intervals=['1w', '3d','2d', '1d', '12h', '6h', '4h'] # '30m', '15m'
#intervals=['4h']





sheet_output=f'Simulations_summaries- {start_date}.xlsx'
#strategy='0.4'
intervals=['1w', '3d','2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']

start_date = pd.to_datetime(start_date)


if os.path.exists(f"{sheet_output}.xlsx"):
    summary_df = pd.read_excel(f"{sheet_output}.xlsx")
    #summary_df = summary_df[summary_df['Strategy'] != 0.3] # usuwam strategy =- 0.3
    print(f"✅ Wczytano istniejący plik: {sheet_output}.xlsx")
else:
    summary_df = pd.DataFrame(columns=ordered_cols)
    print(f"📄 Rozpoczynamy nowy plik: {sheet_output}.xlsx")
extra_cols= ['Start_date', 'End_date', 'length(d)', 'ROI(% year)', 'Deposit', 'Category', 'Okres']
for col in extra_cols:
    if col not in summary_df.columns:
        summary_df[col] = None  # uzupełnij brakujące kolumny

#os.makedirs(folder_output ,exist_ok=True)
for interval in intervals:
    
    for idx, row in strategies.iterrows():
        strategy = row['Numer']
        strategy=str(strategy)
        description = row['Wskaźniki']
        

        for index, row in dane_gieldowe.iterrows():

            ticker = row['Ticker']
            ticker_plik = row['Plik']
            category = row["Category"]
            exists = summary_df[
                (summary_df['Strategy'] == float(strategy)) &
                (summary_df['Description'] == description) &
                (summary_df['Ticker'] == ticker) &
                (summary_df['Interval'] == interval)
            ]
            if not exists.empty:
                if not any(pd.isnull(exists.iloc[0][col]) for col in extra_cols) and False :
                    
                    #if str(strategy) != "0.3":

                    # history_path = f'{start_date = pd.to_datetime(start_date)}/{ticker}- strategy {strategy}, interval {interval} - history .xlsx'
                    # history_df = pd.read_excel(history_path)
                    # idx = exists.index[0]
                    # start_date = history_df['timestamp'].iloc[0]
                    # end_date = history_df['timestamp'].iloc[-1]
                    # initial_value = history_df['account_value'].iloc[0]
                    # summary_df=upload_row(summary_df, idx, start_date, end_date, initial_deposit)
                    # print(f"ZAKTUALIZOWANO {ticker}, {interval}")
                    continue
                    print(strategy)
                else:
                    history_path = f'{output_path}/{ticker}- strategy {strategy}, interval {interval} - history .xlsx'
                    history_df = pd.read_excel(history_path)
                    idx = exists.index[0]
                    start_date = history_df['timestamp'].iloc[0]
                    end_date = history_df['timestamp'].iloc[-1]
                    initial_value = history_df['account_value'].iloc[0]
                    summary_df=upload_row(summary_df, idx, start_date, end_date, initial_value, category, okres_id)
                    print(f"ZAKTUALIZOWANO {ticker}, {interval}")
                continue  # już dodane, pomiń

            history_path = f'{output_path}/{ticker}- strategy {strategy}, interval {interval} - history .xlsx'
            trades_path = f'{output_path}/{ticker}- strategy {strategy}, interval {interval} - trades .xlsx'
        
# Sprawdzenie, czy pliki istnieją
            if not os.path.exists(history_path) or not os.path.exists(trades_path):
                #print(f"⚠️ Pominięto: brak plików dla {ticker} / strategy {strategy} / interval {interval}")
                continue  # Pomijamy tę iterację pętli
            start_date=pd.to_datetime(row['Początek'])
            end_date=pd.to_datetime(row['Koniec'])
            # Wczytanie danych
            history_df = pd.read_excel(history_path)
            trades_df = pd.read_excel(trades_path)
            if(len(trades_df)<1):
                continue
            start_date = history_df['timestamp'].iloc[0]
            end_date = history_df['timestamp'].iloc[-1]
            initial_value = history_df['account_value'].iloc[0]
            new_row=create_new_row(strategy,description, ticker, interval, generate_simulation_summary(trades_df, history_df), start_date, end_date,  initial_value, category, okres_id)
            print(f"DODANO {ticker}, {interval}")
            summary_df = pd.concat([summary_df, new_row], ignore_index=True)  


summary_df.to_excel(sheet_output, index=False)

📄 Rozpoczynamy nowy plik: Simulations_summaries- 2023-08-01.xlsx.xlsx
DODANO 1INCH, 1w
DODANO AAVE, 1w
DODANO ACM, 1w
DODANO ADADOWN, 1w
DODANO ADAUP, 1w
DODANO ADA, 1w
DODANO AIXBT, 1w
DODANO AKRO, 1w


C:\Users\Arek\AppData\Local\Temp\ipykernel_48172\2322716319.py:149: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_df = pd.concat([summary_df, new_row], ignore_index=True)


DODANO ALGO, 1w
DODANO ALICE, 1w
DODANO ALPACA, 1w
DODANO ALPHA, 1w
DODANO ANKR, 1w
DODANO ANT, 1w
DODANO API3, 1w
DODANO ARB, 1w
DODANO ARDR, 1w
DODANO ARPA, 1w
DODANO AR, 1w
DODANO ASR, 1w
DODANO ATA, 1w
DODANO ATM, 1w
DODANO ATOM, 1w
DODANO AUCTION, 1w
DODANO AUDIO, 1w
DODANO AVA, 1w
DODANO AVAX, 1w
DODANO AXS, 1w
DODANO BADGER, 1w
DODANO BAKE, 1w
DODANO BAL, 1w
DODANO BAND, 1w
DODANO BAR, 1w
DODANO BAT, 1w
DODANO BCH, 1w
DODANO BEL, 1w
DODANO BERA, 1w
DODANO BLZ, 1w
DODANO BNBDOWN, 1w
DODANO BNBUP, 1w
DODANO BNB, 1w
DODANO BNT, 1w
DODANO BOND, 1w
DODANO BONK, 1w
DODANO BTCDOWN, 1w
DODANO BTCUP, 1w
DODANO BTC, 1w
DODANO BTS, 1w
DODANO BURGER, 1w
DODANO BUSD, 1w
DODANO C98, 1w
DODANO CAKE, 1w
DODANO CELO, 1w
DODANO CELR, 1w
DODANO CFX, 1w
DODANO CHR, 1w
DODANO CHZ, 1w
DODANO CKB, 1w
DODANO CLV, 1w
DODANO COCOA, 1w
DODANO COFFEE, 1w
DODANO COMP, 1w
DODANO COOKIE, 1w
DODANO COS, 1w
DODANO COTI, 1w
DODANO CRV, 1w
DODANO CTK, 1w
DODANO CTSI, 1w
DODANO CTXC, 1w
DODANO CVC, 1w
DODANO DASH,